## PuLPとGurobi/Pythonによる線形最適化問題のモデリング


久保幹雄　(東京海洋大学）

* 線形最適化ラッパー mypulp
* 例題と双対最適解
* 輸送問題，多品種輸送問題
* 栄養問題
* 定式化のコツ
* Gurobiのパラメータ
* ベンチマーク比較



### 線形最適化ソルバー(モデラー）
-----------


* ### Gurobi（商用，アカデミックフリー）のソルバー
  * 独自のPythonインターフェイス（「あたらしい数理最適化」（近代科学社）で採用）
 * 凸2次（制約）整数，2次錐最適化
* ### PuLP （MITライセンス）のモデラー
  * メインソルバーはCBC(COIN Branch & Cut; EPLライセンス），その他様々なソルバーと連携(MPSフォーマット経由）





### 共通Model $\Rightarrow$ Wrapper $\Rightarrow$ Modeler  $\Rightarrow$ Solver

 <div align="center">
<img src="wrapper.jpg" alt="Drawing" style="width: 700px;"/>





## 例題　

* トンコケ，コケトン，ミックスの丼を販売
* 資源制約の下で，利益を最大化

 <div align="center">
<img src="LP.jpg" alt="Drawing" style="width: 600px;"/>


変数

* トンコケ丼 $x_1$，コケトン丼 $x_2$，ミックス丼 $x_3$

* 定式化

\begin{array}{l c c c c c}
 \mbox{maximize} & 15 x_1  & + 18 x_2 & +30 x_3 &      & \\
 \mbox{subject to}   & 2x_1   & + x_2   & + x_3   & \leq & 60 \\
               &  x_1   & + 2 x_2 & + x_3  &\leq  & 60 \\
               &        &         &   x_3  &\leq  & 30 \\
               &        &         & x_1,x_2,x_3 & \geq & 0 
 \end{array}

In [18]:
#from gurobipy import *
from mypulp import *
model = Model('lo1')

x1 = model.addVar(name='x1')
x2 = model.addVar(name='x2')
x3 = model.addVar(ub=30.0, name='x3')

model.update() #Gurobiの怠惰な更新(lazy update)という仕様（忘れずに！）
    
model.addConstr(2*x1 + x2 + x3 <= 60)
# 別の定義方法 1
#L1 = LinExpr([2,1,1],[x1,x2,x3]) #線形表現(式）
# 別の定義方法 2
#L1 = LinExpr()     #線形表現(式）
#L1.addTerms(2,x1)  #項を追加
#L1.addTerms(1,x2)
#L1.addTerms(1,x3)
#model.addConstr(lhs=L1,sense='<',rhs=60)  #制約を追加

model.addConstr(x1 + 2*x2 + x3 <= 60)

model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)

model.optimize()

if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)

Opt. Value= 1230.0
x1 10.0
x2 10.0
x3 30.0


## 基本クラスと主要メソッド

 <div align="center">
<img src="solver.jpg" alt="Drawing" style="width: 500px;"/>


### モデルファイルの出力
------------
* model.write('ファイル名.lp')で **LPフォーマット**(Linear Programming (LP) format)で保存

* model.write('ファイル名.mps')で**MPSフォーマット** (Mathematical Programming System (MPS) format) で保存
  * 可読性はないが，ほとんどの最適化ソルバーが対応している古典的な書式
  
* 注意：Gurobiの場合にはmodel.update()を直前にするのを忘れずに

* PuLPだと print(model) でも画面にLPフォーマットを出力


In [2]:
model.write('lo1.lp')
print(model)
model.write('lo1.mps')
print('MPS file =========================')
!type lo1.mps

lo1:
MAXIMIZE
15*x1 + 18*x2 + 30*x3 + 0
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

c_2: x1 + 2 x2 + x3 <= 60

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 30 Continuous

MPS file =========================
*SENSE:Maximize
NAME          lo1
ROWS
 N  OBJ
 L  c_1
 L  c_2
COLUMNS
    x1        c_1        2.000000000000e+00
    x1        c_2        1.000000000000e+00
    x1        OBJ        1.500000000000e+01
    x2        c_1        1.000000000000e+00
    x2        c_2        2.000000000000e+00
    x2        OBJ        1.800000000000e+01
    x3        c_1        1.000000000000e+00
    x3        c_2        1.000000000000e+00
    x3        OBJ        3.000000000000e+01
RHS
    RHS       c_1        6.000000000000e+01
    RHS       c_2        6.000000000000e+01
BOUNDS
 UP BND       x1         1.000000000000e+100
 UP BND       x2         1.000000000000e+100
 UP BND       x3         3.000000000000e+01
ENDATA


## 双対問題
-----------

### 資源（豚肉，鶏肉，牛肉）100グラムの価値を推定

主問題
------------

\begin{array}{l c c c c c}
 \mbox{maximize} & 15 x_1  & + 18 x_2 & +30 x_3 &      & \\
 \mbox{subject to}   & 2x_1   & + x_2   & + x_3   & \leq & 60 \\
               &  x_1   & + 2 x_2 & + x_3  &\leq  & 60 \\
               &        &         &   x_3  &\leq  & 30 \\
               &        &         &x_1,x_2,x_3  & \geq & 0 
 \end{array}



双対問題
--------------

\begin{array}{l c c c c c}
 \mbox{minimize} & 60 \pi_1 & + 60 \pi_2& +30 \pi_3 &      & \\
 \mbox{subject to}   & 2\pi_1   & + \pi_2   &         & \geq & 15 \\
               &  \pi_1   & + 2\pi_2  &         &\geq  & 18 \\
               &  \pi_1   & +\pi_2    &  +\pi_3   &\geq  & 30 \\
               &          &           & \pi_1,\pi_2,\pi_3  & \geq & 0 
 \end{array}
 
最適双対変数は 4, 7, 19 

豚肉は百グラム$400$ 円，鶏肉は百グラム $700$ 円， 牛肉は百グラム
$1900$ 円の価値をもつ

In [3]:
for c in model.getConstrs():
    print( c.ConstrName, c.Pi )

c_1 4.0
c_2 7.0


## 輸送問題
---------------

**quicksum**と**multidict**を用いた一般的な記述法

|顧客 $i$ | 1  |  2  | 3  | 4 | 5 | 
|--:|----:|----:|---:|---:|---:|
|需要量 $d_i$  |80 | 270 | 250 | 160 | 180 |   
|工場 $j$|  輸送費用 | $c_{ij}$ |   |  |   | 容量 $M_j$  |
|1      | 4 | 5 | 6 | 8 | 10 |  500 |
|2      | 6  |4 | 3 | 5 | 8 |  500 |
|3      | 9  | 7 | 4 | 3 | 4 |  500 |




$x_{ij}= \mbox{工場 $j$ から顧客 $i$ に輸送される量} $

定式化

\begin{array}{l l l} 
\nonumber
 \mbox{ minimize } & \displaystyle\sum_{i \in I} \displaystyle\sum_{j \in J} c_{ij} x_{ij}  &     \\
 \mbox{ subject to   } &
\nonumber
 \displaystyle\sum_{j \in J} x_{ij} =d_i &  \forall  i \in I  \\
\nonumber
   & \displaystyle\sum_{i \in I} x_{ij} \leq M_j &  \forall  j \in J \\   
\nonumber
                 & x_{ij} \geq 0 & \forall  i \in I; j \in J  
\end{array}

In [4]:
# multidict の使用法
from mypulp import *
name, height, weight=multidict({'Taro':[145,30],'Hanako':[138,34],'Simon':[150,45]})
print(name)
print(height)
print(weight)

['Taro', 'Simon', 'Hanako']
{'Taro': 145, 'Simon': 150, 'Hanako': 138}
{'Taro': 30, 'Simon': 45, 'Hanako': 34}


In [5]:
# quicksum の使用法
from mypulp import *
model=Model()
a=[5,4,2]
x=[model.addVar() for i in range(3)]
L = quicksum(a[i]*x[i] for i in range(3))
print(L)

5*x_1 + 4*x_2 + 2*x_3


In [6]:
#from gurobipy import *
from mypulp import *
I,d = multidict({1:80, 2:270, 3:250 , 4:160, 5:180}) # demand
J,M = multidict({1:500, 2:500, 3:500})               # capacity
c = {(1,1):4,    (1,2):6,    (1,3):9,  # cost
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):4,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4,
     }

model = Model('transportation')
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(vtype='C', name='x({0},{1})'.format(i,j))
model.update()

for i in I:
    model.addConstr(quicksum(x[i,j] for j in J if (i,j) in x) == d[i],
                    name='Demand({0})'.format(i))
for j in J:
    model.addConstr(quicksum(x[i,j] for i in I if (i,j) in x) <= M[j], 
                    name='Capacity({0})'.format(j))
model.setObjective(quicksum(c[i,j]*x[i,j]  for (i,j) in x), GRB.MINIMIZE)

model.optimize()
print( 'Optimal value:', model.ObjVal)

EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print('{0:>5} from factory {1:>2} to customer {2:>2}'.format(x[i,j].X,j,i) )
        
print ('{0:>15}: {1:>8} , {2:>4}'.format('Const. Name', 'Slack', 'Dual'))
for c in model.getConstrs():
    print ('{0:>15}: {1:>8} , {2:>4}'.format(c.ConstrName,c.Slack,c.Pi))

Optimal value: 3370.0
230.0 from factory  2 to customer  3
 20.0 from factory  3 to customer  3
160.0 from factory  3 to customer  4
270.0 from factory  2 to customer  2
 80.0 from factory  1 to customer  1
180.0 from factory  3 to customer  5
    Const. Name:    Slack , Dual
      Demand(1):     -0.0 ,  4.0
      Demand(2):     -0.0 ,  5.0
      Demand(3):     -0.0 ,  4.0
      Demand(4):     -0.0 ,  3.0
      Demand(5):     -0.0 ,  4.0
    Capacity(1):    420.0 ,  0.0
    Capacity(2):     -0.0 , -1.0
    Capacity(3):    140.0 ,  0.0


## 多品種輸送問題
---------------
 疎な問題の扱い方と ** tuplelist **

$
  x_{ijk}= \mbox{工場 $j$ から顧客 $i$ に製品 $k$ が輸送される量}
$

 工場1では製品2,4を，工場2では製品1,2,3を，工場3では製品2,3,4を製造可能

`produce = {1:[2,4], 2:[1,2,3], 3:[2,3,4]}`  


定式化

\begin{array}{l l l} 
\nonumber
 \mbox{ minimize } & \displaystyle\sum_{i \in I} \displaystyle\sum_{j \in J} 
 \displaystyle\sum_{k \in K} c_{ijk} x_{ijk}  &     \\
 \mbox{ subject to   } &
\nonumber
 \displaystyle\sum_{j \in J} x_{ijk} =d_{ik} &  \forall  i \in I; k \in K  \\
\nonumber
   & \displaystyle\sum_{i \in I} \displaystyle\sum_{k \in K} x_{ijk} \leq M_j &  \forall  j \in J  \\   
\nonumber
                 & x_{ijk} \geq 0 & \forall  i \in I; j \in J; k \in K  
\end{array}

In [7]:
# tuplelist の使用法
from mypulp import *
T = tuplelist([('Sara','Apple'),('Taro','Pear'),('Jiro','Orange'),('Simon','Apple')])
print( T.select('*','Apple') )               

[('Sara', 'Apple'), ('Simon', 'Apple')]


In [8]:
from mypulp import *
d = {(1,1):80,   (1,2):85,   (1,3):300,  (1,4):6,
     (2,1):270,  (2,2):160,  (2,3):400,  (2,4):7,
     (3,1):250,  (3,2):130,  (3,3):350,  (3,4):4,
     (4,1):160,  (4,2):60,   (4,3):200,  (4,4):3,
     (5,1):180,  (5,2):40,   (5,3):150,  (5,4):5
     }

I = set([i for (i,k) in d])
K = set([k for (i,k) in d])
J,M = multidict({1:3000, 2:3000, 3:3000})  # capacity
produce = {1:[2,4], 2:[1,2,3], 3:[2,3,4]}  # products that can be produced in each facility
weight = {1:5, 2:2, 3:3, 4:4}              # {commodity: weight<float>}

cost = {(1,1):4,  (1,2):6, (1,3):9,        # {(customer,factory): cost<float>}
        (2,1):5,  (2,2):4, (2,3):7,
        (3,1):6,  (3,2):3, (3,3):4,
        (4,1):8,  (4,2):5, (4,3):3,
        (5,1):10, (5,2):8, (5,3):4
        }

c = {}
for i in I:
    for j in J:
        for k in produce[j]:
            c[i,j,k] = cost[i,j] * weight[k]            
model = Model('multi-commodity transportation')

x = {}
for (i,j,k) in c:
    x[i,j,k] = model.addVar(vtype='C', name='x({0},{1},{2})'.format(i,j,k))
model.update()
arcs = tuplelist([(i,j,k) for (i,j,k) in x])
for i in I:
    for k in K:
        model.addConstr(quicksum(x[i,j,k] for (i,j,k) in arcs.select(i,'*',k)) == d[i,k],
                        'Demand({0},{1})'.format(i,k))
for j in J:
    model.addConstr(quicksum(x[i,j,k] for (i,j,k) in arcs.select('*',j,'*')) <= M[j],
                    'Capacity({0})'.format(j))

model.setObjective(quicksum(c[i,j,k]*x[i,j,k]  for (i,j,k) in x), GRB.MINIMIZE)
model.optimize()
print ('Optimal value:',model.ObjVal)

EPS = 1.e-6
for (i,j,k) in x:
    if x[i,j,k].X > EPS:
        print ('{0:>5}s units {1:>2} from {2:>2} to {3:>2}'.format(x[i,j,k].X,k,j,i))

Optimal value: 43536.0
  3.0s units  4 from  3 to  4
 80.0s units  1 from  2 to  1
250.0s units  1 from  2 to  3
  7.0s units  4 from  1 to  2
 60.0s units  2 from  3 to  4
300.0s units  3 from  2 to  1
150.0s units  3 from  3 to  5
200.0s units  3 from  3 to  4
 40.0s units  2 from  3 to  5
400.0s units  3 from  2 to  2
160.0s units  2 from  2 to  2
270.0s units  1 from  2 to  2
130.0s units  2 from  2 to  3
160.0s units  1 from  2 to  4
 85.0s units  2 from  1 to  1
  5.0s units  4 from  3 to  5
  4.0s units  4 from  3 to  3
350.0s units  3 from  2 to  3
  6.0s units  4 from  1 to  1
180.0s units  1 from  2 to  5


### 栄養問題（実行不可能性の取り扱い）
----------------

|栄養素 $N$ | Cal | Carbo | Protein | VitA | VitC | Calc | Iron |価格　|
|--:|----:|----:|---:|---:|---:|---:|---:|---:|
|商品名 $F$ | |  | $n_{ij}$  |  |  | |  |$c_j$　
| CQPounder | 556| 39| 30| 147| 10| 221| 2.4| 360|
|Big M | 556| 46| 26| 97 | 9 | 142| 2.4| 320|
|FFilet | 356| 42| 14| 28 | 1 | 76 | 0.7| 270|
|Chicken | 431| 45| 20| 9 | 2 | 37 | 0.9| 290|
|Fries| 249| 30| 3 | 0 | 5 | 7 | 0.6| 190|
|Milk | 138| 10| 7 | 80 | 2 | 227| 0 | 170|
|VegJuice | 69 | 17| 1 | 750| 2 | 18 | 0 | 100|
|上限 $a_i$ | 3000| 375| 60| 750| 100| 900| 7.5|
|下限 $b_i$ | 2000| 300| 50| 500| 85| 660| 6.0|




定式化 

$x_j$ は商品 $j$ の購入量(実数）

\begin{array}{l l l}
   \mbox{minimize}    & \displaystyle\sum_{j \in F} c_j x_j          &  \\
   \mbox{subject to}  & a_i \leq \displaystyle\sum_{j \in F} n_{ij} x_j \leq b_i &  i \in N    \\
                      & x_j \geq 0  &     j \in F
\end{array}

$d_i$: 不足変数

$s_i$: 超過変数



改良した制約

$$a_i - d_i \leq \displaystyle\sum_{j \in F} n_{ij} x_j \leq b_i +s_i  \ \   i \in N$$

変更した目的関数（Mは大きな数）

\begin{array}{l l l}
   \mbox{minimize}    & \displaystyle\sum_{j \in F} c_j x_j  + \displaystyle\sum_{i \in N} M (d_i+s_i)  &  
\end{array}

In [39]:
F, c, n = multidict({      
    'CQPounder':  [ 360, {'Cal':556, 'Carbo':39, 'Protein':30, 'VitA':147,
                          'VitC': 10, 'Calc':221, 'Iron':2.4}], 
    'Big M'    :  [ 320, {'Cal':556, 'Carbo':46, 'Protein':26, 'VitA':97,
                          'VitC':  9, 'Calc':142, 'Iron':2.4}], 
    'FFilet'   :  [ 270, {'Cal':356, 'Carbo':42, 'Protein':14, 'VitA':28, 
                          'VitC':  1, 'Calc': 76, 'Iron':0.7}], 
    'Chicken'  :  [ 290, {'Cal':431, 'Carbo':45, 'Protein':20, 'VitA': 9, 
                          'VitC':  2, 'Calc': 37, 'Iron':0.9}], 
    'Fries'    :  [ 190, {'Cal':249, 'Carbo':30, 'Protein': 3, 'VitA': 0, 
                          'VitC':  5, 'Calc':  7, 'Iron':0.6}], 
    'Milk'     :  [ 170, {'Cal':138, 'Carbo':10, 'Protein': 7, 'VitA':80,
                          'VitC':  2, 'Calc':227, 'Iron': 0}], 
    'VegJuice' :  [ 100, {'Cal': 69, 'Carbo':17, 'Protein': 1, 'VitA':750,
                          'VitC':  2, 'Calc':18,  'Iron': 0}] })
N, a, b = multidict({       
    'Cal'     : [ 2000,  3000],
    'Carbo'   : [  300,  375 ],
    'Protein' : [   50,   60 ],
    'VitA'    : [  500,  750 ],
    'VitC'    : [   85,  100 ],
    'Calc'    : [  660,  900 ],
    'Iron'    : [  6.0,  7.5 ]})
model = Model('modern diet')
x,s,d = {},{},{}
for j in F:
    x[j] = model.addVar(vtype='C', name='x({0})'.format(j))
for i in N:
    s[i] = model.addVar(vtype='C', name='surplus({0})'.format(i))
    d[i] = model.addVar(vtype='C', name='deficit({0})'.format(i))
model.update()
for i in N:
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) >= a[i]-d[i],'NutrLB({0})'.format(i))
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) <= b[i]+s[i],'NutrUB({0})'.format(i))
model.setObjective(quicksum(c[j]*x[j]  for j in F)+
                   quicksum(9999*d[i]+9999*s[i] for i in N), GRB.MINIMIZE )
model.optimize()
status = model.Status
if status == GRB.Status.OPTIMAL:
    print ("Optimal value:",model.ObjVal)
    for j in F:
        if x[j].X > 0:
            print (j,x[j].X)
    for i in N:
        if d[i].X > 0:
            print ('deficit of {0} ={1}'.format(i,d[i].X))
        if s[i].X > 0:
            print ('surplus of {0} ={1}'.format(i,s[i].X))

Optimal value: None
deficit of Cal =2000.0
deficit of Calc =660.0
deficit of VitC =85.0
deficit of Iron =6.0
deficit of VitA =500.0
deficit of Carbo =300.0
deficit of Protein =50.0


## 定式化のコツ
------------ 

* ### 疎性の考慮
  * データ構造としては「辞書」がおすすめ
  * 例：大規模輸送問題
    * 緯度・経度からx,y座標を計算（Basemapモジュールなど）
    * SciPyの計算幾何モジュール spatial で近い点のみ距離行列計算
    * 使用する変数を近い点同士に限定して求解


* ### 実行不可能性に注意
  * 幾つかの制約をソフトに（逸脱を許すように）しておく
    

* ### 最大値の最小化
  * $3x_1+4x_2$ と $2x_1+7x_2$ の大きい方を小さくしたい 
    * 変数 $z$ を導入し，$3x_1 + 4x_2  \leq z, 2x_1 + 7x_2  \leq z$ 加えた後で， $z$ を最小化
    * 最小値の最大化も同様
  * 注意 1：最小値の最小化（最大値の最大化）は難しい（0-1変数が必要；整数最適化で解説）
  * 注意 2: 整数が入るとこの方法は効率が悪い; 例：最大完了時刻の最小化（2分探索などを推奨）
    


* ### 絶対値の最小化
  * 例：実数変数 $x$ の絶対値 $|x|$ を最小化したい
    * 2 つの新しい非負の実数変数 $y$ と $z$ を導入
    * $x=y-z$ を追加
    * $|x|$ をすべて $y+z$ で置き換え

  * 別の方法：
    * $|x|$ を新しい変数 $z$ で置き換え
    * $z \geq x$ と $z \geq -x$ の制約を追加
   
  * 注意 ：絶対値の最大化は難しい（0-1変数が必要）

In [5]:
#ジュースの販売
#from gurobipy import *
from mypulp import *
model = Model()
xA = model.addVar(name='xA')
xB = model.addVar(name='xB')
model.update() 

model.addConstr(3*xA + 2*xB  <= 200)  #grape upper bound
model.addConstr(  xA + 2*xB  <= 100)  #apple upper bound

model.setObjective(3*xA + 4*xB, GRB.MAXIMIZE)

model.optimize()

if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)

Opt. Value= 250.0
xA 50.0
xB 25.0


In [8]:
# Easy Puzzle 
#from gurobipy import *
from mypulp import *
model = Model()
xA = model.addVar(name='xA')
xB = model.addVar(name='xB')
xC = model.addVar(name='xC')
model.update() 

model.addConstr(xA == xB +200 ) 
model.addConstr(xB == xC +600 )  

model.setObjective(xA +xB +xC, GRB.MINIMIZE)

model.optimize()

if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)

Opt. Value= 1400.0
xA 800.0
xB 600.0
xC 0.0


In [13]:
#裕一郎君
#from gurobipy import *
from mypulp import *
model = Model()
x1 = model.addVar(name='x1') #run
x2 = model.addVar(name='x2') #walk
x3 = model.addVar(name='x3') #rest
model.update() 

model.addConstr( x1+x2+x3== 360+40)
model.addConstr( 180*x1+ 70*x2 == 42.195*1000)
model.addConstr( x2 ==2*x1)
model.setObjective(x1, GRB.MAXIMIZE)

model.optimize()

if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)

Opt. Value= 131.85938
x1 131.85938
x2 263.71875
x3 4.421875


In [21]:
#丼チェーン店長の悩み（改）
#from gurobipy import *
from mypulp import *
model = Model('lo1')

x1 = model.addVar(name='x1')
x2 = model.addVar(name='x2')
x3 = model.addVar(name='x3')
x4 = model.addVar(ub=10.0, name='x4')
model.update() 

model.addConstr(x1 + 2*x2 + x3 <= 90)
model.addConstr(2*x1 + x2 + x3 <= 90)
model.addConstr(x3 + 300*x4 <= 60)

model.setObjective(15*x1 + 18*x2 + 30*x3 +50*x4, GRB.MAXIMIZE)

model.optimize()

if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)
    print('Dual Price')    
    for c in model.getConstrs():
        print (c.Pi)
    

Opt. Value= 2130.0
x1 10.0
x2 10.0
x3 60.0
x4 0.0
Dual Price
7.0
4.0
19.0


In [44]:
#倉庫経由の輸送問題
from mypulp import *
Customer,d = multidict({1:80, 2:270, 3:250 , 4:160, 5:180}) # demand
Plant,M = multidict({1:500, 2:500, 3:500})                 # capacity
C = {(1,1):1,    (1,2):6,  (1,3):3,  # C[j,k] : Transportation cost from plant k to warehouse j   
     (2,1):3,    (2,2):1,  (2,3):2  
     }
c = {(1,1):4,    (1,2):6,    #c[i,j] : Transportation cost from warehouse j to customer i 
     (2,1):5,    (2,2):4,  
     (3,1):6,    (3,2):3,    
     (4,1):8,    (4,2):5,   
     (5,1):10,   (5,2):8
     }
Warehouse =list(range(1,3))

model = Model('transshipment')
x, X = {}, {} 
for i in Customer:
    for j in Warehouse:
        x[i,j] = model.addVar(vtype='C', name='x({0},{1})'.format(i,j))
for j in Warehouse:
    for k in Plant:
        X[j,k] = model.addVar(vtype='C', name='X({0},{1})'.format(j,k))
model.update()

for i in Customer:
    model.addConstr(quicksum(x[i,j] for j in Warehouse ) == d[i],
                    name='Demand({0})'.format(i))
for k in Plant:
    model.addConstr(quicksum(X[j,k] for j in Warehouse )<= M[k], 
                    name='Capacity({0})'.format(k))

for j in Warehouse:
    model.addConstr( quicksum(X[j,k] for k in Plant) ==
                     quicksum(x[i,j] for i in Customer ),
                   name='FlowConserve({0})'.format(j))
                            
model.setObjective(quicksum(C[j,k]*X[j,k]  for (j,k) in X) +quicksum(c[i,j]*x[i,j]  for (i,j) in x), GRB.MINIMIZE)

model.optimize()
print( 'Optimal value:', model.ObjVal)

EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print('{0:>5} from warehouse {1:>2} to customer {2:>2}'.format(x[i,j].X,j,i) )

for (j,k) in X:
    if X[j,k].X > EPS:
        print('{0:>5} from factory {1:>2} to warehouse {2:>2}'.format(X[j,k].X,k,j) )
        
print ('{0:>15}: {1:>8} , {2:>4}'.format('Const. Name', 'Slack', 'Dual'))
for c in model.getConstrs():
    print ('{0:>15}: {1:>8} , {2:>4}'.format(c.ConstrName,c.Slack,c.Pi))

Optimal value: 5690.0
250.0 from warehouse  2 to customer  3
180.0 from warehouse  2 to customer  5
160.0 from warehouse  2 to customer  4
 80.0 from warehouse  1 to customer  1
270.0 from warehouse  1 to customer  2
 90.0 from factory  3 to warehouse  2
500.0 from factory  2 to warehouse  2
350.0 from factory  1 to warehouse  1
    Const. Name:    Slack , Dual
      Demand(1):     -0.0 ,  5.0
      Demand(2):     -0.0 ,  6.0
      Demand(3):     -0.0 ,  5.0
      Demand(4):     -0.0 ,  7.0
      Demand(5):     -0.0 , 10.0
    Capacity(1):    150.0 ,  0.0
    Capacity(2):     -0.0 , -1.0
    Capacity(3):    410.0 ,  0.0
FlowConserve(1):     -0.0 ,  1.0
FlowConserve(2): -2.8421709e-14 ,  2.0


In [51]:
#実行不可能性
from mypulp import *
Customer,d = multidict({1:80, 2:270, 3:250 , 4:160, 5:180}) # demand
for i in Customer:
    d[i] *=2
Plant,M = multidict({1:500, 2:500, 3:500})                 # capacity
C = {(1,1):1,    (1,2):6,  (1,3):3,  # C[j,k] : Transportation cost from plant k to warehouse j   
     (2,1):3,    (2,2):1,  (2,3):2  
     }
c = {(1,1):4,    (1,2):6,    #c[i,j] : Transportation cost from warehouse j to customer i 
     (2,1):5,    (2,2):4,  
     (3,1):6,    (3,2):3,    
     (4,1):8,    (4,2):5,   
     (5,1):10,   (5,2):8
     }
Warehouse =list(range(1,3))

model = Model('transshipment')
x, X = {}, {} 
S ={}
for i in Customer:
    for j in Warehouse:
        x[i,j] = model.addVar(vtype='C', name='x({0},{1})'.format(i,j))
for j in Warehouse:
    for k in Plant:
        X[j,k] = model.addVar(vtype='C', name='X({0},{1})'.format(j,k))
for k in Plant:
    S[k] =  model.addVar(vtype='C', name='S({0})'.format(k))
model.update()

for i in Customer:
    model.addConstr(quicksum(x[i,j] for j in Warehouse ) == d[i],
                    name='Demand({0})'.format(i))
for k in Plant:
    model.addConstr(quicksum(X[j,k] for j in Warehouse )<= M[k]+S[k] ,
                    name='Capacity({0})'.format(k))

for j in Warehouse:
    model.addConstr( quicksum(X[j,k] for k in Plant) ==
                     quicksum(x[i,j] for i in Customer ),
                   name='FlowConserve({0})'.format(j))
                            
model.setObjective(quicksum(C[j,k]*X[j,k]  for (j,k) in X) +quicksum(c[i,j]*x[i,j]  for (i,j) in x) +
                   quicksum(999999*S[k] for k in Plant), GRB.MINIMIZE)

model.optimize()

print( 'Optimal value:', model.ObjVal)

EPS = 1.e-6
print('Surplus Variables')
for k in Plant:
    print(S[k].X)
print('Flow Variable')
for (i,j) in x:
    if x[i,j].X > EPS:
        print('{0:>5} from warehouse {1:>2} to customer {2:>2}'.format(x[i,j].X,j,i) )

for (j,k) in X:
    if X[j,k].X > EPS:
        print('{0:>5} from factory {1:>2} to warehouse {2:>2}'.format(X[j,k].X,k,j) )
        
print ('{0:>15}: {1:>8} , {2:>4}'.format('Const. Name', 'Slack', 'Dual'))
for c in model.getConstrs():
    print ('{0:>15}: {1:>8} , {2:>4}'.format(c.ConstrName,c.Slack,c.Pi))

Optimal value: 380011120.0
Surplus Variables
0.0
380.0
0.0
Flow Variable
500.0 from warehouse  2 to customer  3
360.0 from warehouse  2 to customer  5
200.0 from warehouse  2 to customer  2
320.0 from warehouse  2 to customer  4
160.0 from warehouse  1 to customer  1
340.0 from warehouse  1 to customer  2
500.0 from factory  3 to warehouse  2
880.0 from factory  2 to warehouse  2
500.0 from factory  1 to warehouse  1
    Const. Name:    Slack , Dual
      Demand(1):     -0.0 , 1000003.0
      Demand(2):     -0.0 , 1000004.0
      Demand(3):     -0.0 , 1000003.0
      Demand(4):     -0.0 , 1000005.0
      Demand(5):     -0.0 , 1000008.0
    Capacity(1):     -0.0 , -999998.0
    Capacity(2):     -0.0 , -999999.0
    Capacity(3):     -0.0 , -999998.0
FlowConserve(1):     -0.0 , 999999.0
FlowConserve(2):     -0.0 , 1000000.0


In [2]:
# Goal keeper's problem
# 混合戦略
from mypulp import *
m=Model()
x ={}
n=2
for i in range(n):
    x[i]=m.addVar(name="x({0})".format(i), vtype="C" )
v=m.addVar(name="value")
m.update()

m.setObjective(v, GRB.MAXIMIZE)
c1=m.addConstr( v<=0.9*x[0]+0.6*x[1])
c2=m.addConstr( v<=0.5*x[0]+0.8*x[1])
m.addConstr( quicksum( x[i] for i in range(n) )==1 )

m.optimize()
print("Obj=",m.ObjVal)
for i in range(n):
    print(x[i].X)
print("Dual Variable=",c1.Pi, c2.Pi)    
    
#Dual Problem (kicker's problem)
m=Model()
x ={}
n=2
for i in range(n):
    x[i]=m.addVar(name="x({0})".format(i), vtype="C" )
v=m.addVar(name="value")
m.update()

m.setObjective(v, GRB.MAXIMIZE)
m.addConstr( v<=0.9*x[0]+0.5*x[1])
m.addConstr( v<=0.6*x[0]+0.8*x[1])
m.addConstr( quicksum( x[i] for i in range(n) )==1 )

m.optimize()
print("Obj=",m.ObjVal)
for i in range(n):
    print(x[i].X)
    
print("Dual Variable=",c1.Pi, c2.Pi)   

Obj= 0.7
0.33333333
0.66666667
Dual Variable= 0.5 0.5
Obj= 0.7
0.5
0.5
Dual Variable= 0.5 0.5


In [3]:
#Dual Problem (kicker's problem)
m=Model()
x ={}
n=2
for i in range(n):
    x[i]=m.addVar(name="x({0})".format(i), vtype="C" )
v=m.addVar(name="value")
m.update()

m.setObjective(v, GRB.MAXIMIZE)
m.addConstr( v<=0.9*x[0]+0.5*x[1])
m.addConstr( v<=0.6*x[0]+0.8*x[1])
m.addConstr( quicksum( x[i] for i in range(n) )==1 )

m.optimize()
print("Obj=",m.ObjVal)
for i in range(n):
    print(x[i].X)

Obj= 0.7
0.5
0.5


In [9]:
#   スーパー配置問題
from mypulp import *
model=Model()

x = [0,2,3,1]
y = [1,0,3,3]

n=len(x)

X=model.addVar(ub=3.0, name ="X")
Y=model.addVar(ub=3.0, name ="Y")
    
a,b={},{}
for i in range(n):
    a[i] =model.addVar(name="a({0})".format(i))
    b[i] =model.addVar(name="b({0})".format(i))
model.update()

for i in range(n):
    model.addConstr( a[i] >= x[i] -X )
    model.addConstr( a[i] >= X-x[i] )
    model.addConstr( b[i] >= y[i] -Y )
    model.addConstr( b[i] >= Y-y[i] )
    
model.setObjective( quicksum( a[i]+b[i] for i in range(n)), GRB.MINIMIZE )

model.optimize()

print(X.X,Y.X)
print(model.ObjVal)
for i in range(n):
    print(a[i].X,b[i].X)

1.0 1.0
9.0
1.0 0.0
1.0 1.0
2.0 2.0
0.0 2.0


In [8]:
#   消防署配置問題
from mypulp import *
model=Model()

x = [0,2,3,1]
y = [1,0,3,3]

n=len(x)

X=model.addVar(ub=3.0, name="X")
Y=model.addVar(ub=3.0, name ="Y")   
Z=model.addVar(name ="Z")

a,b={},{}
for i in range(n):
    a[i] =model.addVar(name="a({0})".format(i))
    b[i] =model.addVar(name="b({0})".format(i))
model.update()

for i in range(n):
    model.addConstr( a[i] >= x[i] -X )
    model.addConstr( a[i] >= X-x[i] )
    model.addConstr( b[i] >= y[i] -Y )
    model.addConstr( b[i] >= Y-y[i] )
    model.addConstr( Z>=a[i]+b[i] )

model.setObjective(Z, GRB.MINIMIZE)

model.optimize()

print(X.X,Y.X)
print(model.ObjVal)
for i in range(n):
    print(a[i].X,b[i].X)

1.5 2.0
2.5
1.5 1.0
0.5 2.0
1.5 1.0
1.5 1.0


### 線形最適化のためのGurobiのパラメータ
--------

> パラメータの変更方法
>*  setParam(“パラメータ名”,値 )
>*  model.Params.パラメータ名 =値



*  **Method** 最適化手法の選択
  * -1 ：自動選択 (既定値）
  * 0  ：主単体法
  * 1  ：双対単体法（分枝ノードの中での既定値）
  * 2　：内点法（2次，2次錐での既定値）
  * 3  ：非決定性並流（線形最適化での既定値）
  * 4  ：決定性並流
  
*  **Threads** 使用するスレッド数の決定

並流最適化：1つのスレッドで双対単体法，残りのスレッドで並列内点法

*  **Crossover** (線形最適化のための）内点法から単体法への移行方法
  * -1 ：自動設定
  * 0 ： クロスオーバーを行わない
  * 1 ：双対変数の固定 => 主変数の固定 => 主単体法
  * 2 ：双対 => 主 =>双対
  * 3 ：主 => 双対 => 主 
  * 4 ：主 => 双対 => 双対 
  

*  **BarHomogenuous**　同次自己双対内点法の選択
  * -1 ：自動選択（分枝限定法のノードで使われたときのみ使用）
  * 0  ：使用しない
  * 1  ：使用する（実行不可能性や非有界の判定をしたいとき）
  
*  **InfUnbInfo** 実行不可能性や非有界の情報を得たいとき
  * 0  ：使用しない(既定値）
  * 1  ：使用する（端線情報 UnbdRayや実行不可能性の証拠 FarkasDual, FarkasProodを得ることができる）


### ベンチマーク比較
http://plato.la.asu.edu/bench.html

* CPLEX-12.6.1  CPLEX
* GUROBI-6.0.0    www.gurobi.com
* MOSEK-7.1.0.9 www.mosek.com
* XPRESS-7.8.0:   XPRESS

アルゴリズム
* S は単体法 (Simplex)
* B は内点法 (Barrier)
* A は自動選択 (Automatic)

### 比較

|幾何平均 |  4.65 | 3.07 | 6.15 | 3.84| 1.43 | 1 |  1.45| 
|--:|----:|----:|----:|----:|----:|----:|----:|
|         | CPXS |  GRBS | MSKS | XPRS | CPXB |  GRBB |  MSKB | 
 


|幾何平均 |  1.10 |  1.93 | 1.05 |  1.73 | 1.38|
|----:|----:|----:|----:|----:|----:|
|     |  XPRB |   CPXA | GRBA | MSKA |  XPRA |
 

## 単体法の比較

* CPLEX-12.6.1    CPLEX
* GUROBI-6.0.0    www.gurobi.com
* MOSEK-7.1.0.24  www.mosek.com
* XPRESS-7.8.0    XPRESS
* CLP-1.16.3      projects.coin-or.org/Clp (with openblas)
* Google-GLOP     LP with Glop
* SOPLEX-2.0.0    soplex.zib.de/
* LP_SOLVE-5.5.2  lpsolve.sourceforge.net/
* GLPK-4.55       www.gnu.org/software/glpk/glpk.html
* MATLAB-R2014b   mathworks.com (dual-simplex)


比較

|   | CPXS|   GRBS |  MSKS |  XPRS |   CLP |  GLOP | SOPLX |  LPSLV |  GLPK |  MATL|
|--:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| 幾何平均 | 1.72 |  1.12 |  2.27 |  1.41 |    1  |  10.8 | 12.3 |  89.8   | 38.9 |  14.1 |

## Questions, Comments, Suggestions?